In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd
from sklearn.linear_model import LinearRegression
np.random.seed(0)

In [2]:
import keras 
import tensorflow as tf
from keras.datasets import cifar10
from keras.callbacks import History 
#using tensorflow backend

Using TensorFlow backend.


In [3]:
def mse_custom(y_true,y_predicted):
    mse_mod = tf.norm(tf.subtract(y_true[3:-3],y_predicted[3:-3]))
    return mse_mod

In [4]:
def new_cases_per_day(fecha_sintomas,graph=False,ret=False):
    """
    args: 
    -fecha_sintomas: day of beginning of symptomps for each case, can't be NaN or NaT (already processed)
    -graph: defaults to False. if True, makes graph of new cases per day 
    -ret: defaults to False. if True, returns days-cases per day  np.array casos_por_dia
    given the days of beginning of symptoms, processes to get np.array of cases per day and days since epidemic
    can graph and return this vector with the defaults explained before
    """
    counts = np.bincount(fecha_sintomas)
    ultima_fecha = max(fecha_sintomas)
    aux = range(ultima_fecha+1)
    casos_por_dia = np.vstack((aux,counts[aux])).T #(days since inicio_epidemia x (fecha_inicio_sintomas == days))
    if graph == True:
        plt.xlabel("Dia")
        plt.ylabel("Casos")
        plt.axvspan(ultima_fecha-10, ultima_fecha+3, facecolor='r', alpha=0.5,label="últimos 10 días")
        plt.plot(casos_por_dia[:,0],casos_por_dia[:,1],'-ob',label="casos hasta dia: "+str(ultima_fecha))
        plt.style.use('ggplot')
        plt.legend()
        #plt.savefig("casos_por_dia_bariloche.pdf")
    if ret == True:
        return casos_por_dia   

In [5]:
def cumulative_cases(fecha_sintomas,graph=False,ret=False):
    """
    args: 
    -fecha_sintomas: day of beginning of symptomps for each case, can't be NaN or NaT (already processed)
    -graph: defaults to False. if True, makes graph of new cases per day 
    -ret: defaults to False. if True, returns days-cumulative cases per day np array
    given the days of beginning of symptoms, processes to get np.array of cumulative cases per day and days since epidemic
    can graph and return this vector with the defaults explained before
    """
    ultima_fecha = max(fecha_sintomas)
    casos_por_dia = new_cases_per_day(fecha_sintomas,ret=True)
    casos_acumulados_por_dia = np.copy(casos_por_dia)
    casos_acumulados_por_dia[:,1] = np.cumsum(casos_por_dia[:,1]) 
    if graph == True:
        plt.xlabel("Dia")
        plt.ylabel("Casos acumulados")
        plt.axvspan(ultima_fecha-10, ultima_fecha+3, facecolor='r', alpha=0.5,label="últimos 10 días")
        plt.scatter(casos_acumulados_por_dia[:,0],casos_acumulados_por_dia[:,1],c='b',label="casos acumulados hasta dia: "+str(ultima_fecha))
        plt.legend()
        plt.style.use('ggplot')
        plt.savefig("casos_acumulados_bariloche.pdf")
    if ret == True:
        return casos_acumulados_por_dia   

In [6]:
def repratio_t_conv(fecha_sintomas,a,graph=False,ret=False):
    """
    args: 
    -fecha_sintomas: day of beginning of symptomps for each case, can't be NaN or NaT (already processed)
    -graph: defaults to False. if True, makes graph of rep ratio per day
    -ret: defaults to False. if True, returns reproductive ratio per day np array of (days,r_days) and
    the n_t vector which is used to calculate it, which is extension of new cases per day made as 
    necessary, by taking means and not linear regression. 
    given the days of beginning of symptoms, processes to get np.array of cumulative cases per day and days since epidemic
    can graph and return this vector with the defaults explained before for days (0,lastday+4)
    conventional version, which means no coef a,b,c,d,e. r_t valid for days >= 6. 
    """
    casos_por_dia = new_cases_per_day(fecha_sintomas,ret=True)
    n_t = np.copy(casos_por_dia) #here we store data + proyection for days t+4 
    prox_dia = max(fecha_sintomas) + 1
    #completing the values for the rest of the values of n_t
    aux = np.zeros(2)
    i = 0
    while i < 4: #completing until the day t+4 bc im gonna need it later for averaging in 7 days
        aux[0] = prox_dia + i
        aux[1] = np.mean(n_t[prox_dia-7:,1])
        n_t  = np.vstack((n_t,aux)) #extending n_t up to day t+1
        i= i +1
    #create storage and calculate values for r_t
    r_t = np.copy(n_t) #here we will store the rt
    r_t = r_t.astype(float)
    dias_aux = np.arange(0,prox_dia+3)
    for i in dias_aux[dias_aux>=6]:
        aux = a[0]*n_t[i-6,1] + a[1]*n_t[i-5,1] + a[2]*n_t[i-4,1] #denominator of the r_t expression
        if(aux==0):
            aux = 1 #the first cases in which the denominator is == 0
        r_t[i,1] = min(((a[3]*n_t[i-1,1] + a[4]*n_t[i,1] + a[5]*n_t[i+1,1])/(aux)),4) #the rest of the days with limit value
    if graph == True:
        plt.style.use('ggplot')
        plt.axvspan(prox_dia-10, prox_dia+3, facecolor='r', alpha=0.5,label="últimos 10 días")
        plt.scatter(r_t[6:prox_dia,0],r_t[6:prox_dia,1],c='b',label=r'$r_t$')
        plt.xlabel("Dia")
        plt.ylabel(r'$r_t$',fontsize=12)
        plt.legend()
        plt.savefig("rt_raw_casos.pdf")
    if ret == True:
        return n_t,r_t

In [7]:
def risk_diagram(fecha_sintomas,pop,a,rep_fun=repratio_t_conv,graph=False,ret=False):
    """
    args: 
    -fecha_sintomas: day of beginning of symptomps for each case, can't be NaN or NaT (already processed)
    -pop: number of inhabitants of the region/population of interest
    -rep_fun: function to calculate empirical reproductive ratio. defaults tp repratio_t_conv (not optimized)
    -graph: defaults to False. if True, makes graph of risk diagram
    -ret: defaults to False. if True, returns the components of risk diagram in two np.arrays, which are
     the attack ratio (days>=20) as well as the rep ratio averaged over 7 days (days>=20). 
    """
    #we proceed to calculate what makes a risk diagram then
    #first, we will calculate r_t average in 7 days
    prox_dia = max(fecha_sintomas) + 1
    n_t,r_t = rep_fun(fecha_sintomas,a,False,True)
    r_t_seven = np.zeros(prox_dia) #real values for index>=9
    a_t = np.zeros(prox_dia) #real values for index >= 13
    i = 9
    while i < prox_dia:
        r_t_seven[i] = np.mean(r_t[i-3:i+4,1])
        i = i+1
    i = 13
    while i < prox_dia:
        a_t[i] = np.sum(n_t[i-13:i+1,1]) 
        i = i+1
    a_t = a_t * (100000/pop)
    if graph == True:
        plt.plot(a_t[13:],r_t_seven[13:],'-or',markersize=5) #not at all fancy risk diagram,looks reasonable
        plt.xlabel(r'$A_{t}^{14}$')
        plt.ylabel(r'$R_{t}^{7}$')
        plt.style.use('ggplot')
        plt.title('Diagrama de riesgo Bariloche',fontsize=10)
        plt.savefig('riskdiagram_brc.pdf')
    if ret == True:
        return r_t_seven,a_t

In [8]:
def sum_last14(casos_por_dia):
    ac_por_dia = np.copy(casos_por_dia)
    for i in casos_por_dia[:,0]:
        index = np.copy(casos_por_dia[casos_por_dia[:,0]<i+1])
        index = index[i-13<=index[:,0]]
        index = index[:,1]
        ac_por_dia[i,1] = np.sum(index) 
    return ac_por_dia

In [9]:
def predict_cases(fecha_sintomas,pop,a,rep_fun=repratio_t_conv,graph=False,ret=False):
    """
    args: 
    -fecha_sintomas: day of beginning of symptomps for each case, can't be NaN or NaT (already processed)
    -pop: number of inhabitants of the region/population of interest
    -rep_fun: function to calculate empirical reproductive ratio. defaults tp repratio_t_conv (not optimized)
    -graph: defaults to False. if True, makes graph of risk diagram
    -ret: defaults to False. if True, returns the predictions and cases per day
    predicts cases per day after calculating the risk diagram, using the empirical reproductive
    ratio given by rep_fun
    """
    prox_dia = max(fecha_sintomas)+1
    casos_por_dia = new_cases_per_day(fecha_sintomas,ret=True)
    r_def,a_t = risk_diagram(fecha_sintomas,pop,a,rep_fun,ret=True)
    r_def = r_def[13:]
    a_t = a_t[13:]
    p_t = r_def * a_t
    valid_days = np.arange(19,prox_dia+6,1)
    casos_por_dia = sum_last14(casos_por_dia) #gets all active cases in the last 14-days 
    if graph == True:
        plt.style.use('ggplot')
        plt.xlabel('Dias desde el comienzo')
        plt.ylabel('Infectados activos en BRC')
        plt.plot(casos_por_dia[:,0],casos_por_dia[:,1]*(100000/pop),'-ob',markersize=4,label="casos hasta dia: "+str(prox_dia))
        plt.plot(valid_days,p_t,linewidth=3,label='prediccion')
        plt.legend()
        #plt.savefig('9oct_predic_casos.pdf')
    if ret == True:
        error_global = np.linalg.norm(casos_por_dia[19:,1]-p_t[:-6])
        return error_global

In [10]:
def change_dates(df):
    #changes fecha_inicio_sintomas according to new criteria
    filt_df1 = (df.fecha_inicio_sintomas.isnull()) #filter fecha_inicio_sintomas = inexistant
    df_sin_fecha = df.loc[filt_df1]
    n_size = df_sin_fecha.shape[0]
    df.loc[filt_df1,"fecha_inicio_sintomas"] = df.loc[filt_df1,"fecha_apertura"] - np.random.randint(0,9,n_size)
    filt_df2 = (df.fecha_inicio_sintomas < 0)
    df[filt_df2] = 0

In [11]:
data = pd.read_csv("casos/CasosCovid19_arg.txt",sep=",",quotechar='"',
                   parse_dates=["fecha_inicio_sintomas","fecha_apertura"],na_values=['']) #data loading
df = pd.DataFrame(data) #converting to dataframe for use

In [12]:
filt_df1 = (df.fecha_inicio_sintomas.notnull()) #filter fecha_inicio_sintomas = inexistant
inicio_epidemia = min(df.loc[filt_df1,"fecha_inicio_sintomas"]) #first symptoms of a person registered
ultima_actualizacion_sintomas = max(df.loc[filt_df1,"fecha_inicio_sintomas"]) #last day symptoms of a person registered
ultima_actualizacion_apertura = max(df["fecha_apertura"])
df["fecha_inicio_sintomas"] -= inicio_epidemia #correcting by inicio_epidemia 
df["fecha_apertura"] -= inicio_epidemia #correcting by inicio_epidemia
df.fecha_inicio_sintomas = df.fecha_inicio_sintomas.dt.days #change to int, ditch days 
df.fecha_apertura = df.fecha_apertura.dt.days #change to int, ditch days
change_dates(df) #replaces non existing fecha_inicio_sintomas acc to new criteria
print("primer sintoma de persona confirmada: ",inicio_epidemia)
print("ultimo sintoma de persona confirmada: ",ultima_actualizacion_sintomas)
print("ultima apertura de persona confirmada: ",ultima_actualizacion_apertura)

primer sintoma de persona confirmada:  2020-01-01 00:00:00
ultimo sintoma de persona confirmada:  2020-12-29 00:00:00
ultima apertura de persona confirmada:  2020-12-29 00:00:00


In [13]:
fecha_sintomas = df.fecha_inicio_sintomas.to_numpy() #numpy array of fecha_inicio_sintomas
fecha_apertura = df.fecha_apertura.to_numpy() #numpy array of fecha_apertura
fecha_sintomas = fecha_sintomas.astype(int)

In [14]:
aw = np.zeros(6) + 1
pop = 100000
rep_fun = repratio_t_conv
r_def,a_t = risk_diagram(fecha_sintomas,pop,aw,rep_fun,ret=True) #hasta aca perfecto
a_t = a_t[10:] 
a = a_t[:-4]

In [15]:
casos_por_dia = new_cases_per_day(fecha_sintomas,ret=True)
casos_por_dia = sum_last14(casos_por_dia)
casos_por_dia = casos_por_dia[17:,1]
b = np.zeros(casos_por_dia.shape[0]+3,dtype=float)
b[:casos_por_dia.shape[0]] = casos_por_dia

In [16]:
casos_por_dia = new_cases_per_day(fecha_sintomas,ret=True)
n_t = np.copy(casos_por_dia)
prox_dia = max(fecha_sintomas) + 1
#completing the values for the rest of the values of n_t
aux = np.zeros(2)
i = 0
while i < 4: #completing until the day t+4 bc im gonna need it later for averaging in 7 days
    aux[0] = prox_dia + i
    aux[1] = np.mean(n_t[prox_dia-7:,1])
    n_t  = np.vstack((n_t,aux)) #extending n_t up to day t+1
    i= i +1
dias = np.copy(n_t[:,0]).astype(int)
dias = dias[dias>=10]
dias = dias[dias<=max(fecha_sintomas)-4]
m1 = np.zeros((dias.shape[0],3)) #denominator of r7 expression
m2 = np.zeros((dias.shape[0],3)) #numerator of r7 expression

In [17]:
for i in dias:
    m1[i-10,:] = n_t[i-6:i-3,1]
    m2[i-10,:] = n_t[i-1:i+2,1]

In [18]:
n_dias = m1.shape[0]

In [19]:
def my_matrix(shape):
    m = np.zeros((shape,shape))
    i = 3
    while i < shape-3:
        m[i,i-3:i+4] = 1/7
        i = i + 1
    return m

In [20]:
def ret_matrix(shape):
    numpyA = my_matrix(shape)
    numpyA = keras.backend.variable(numpyA)
    mat = numpyA
    return mat

In [21]:
n_dias = m1.shape[0]

In [22]:
mat = ret_matrix(n_dias)

In [ ]:
input_1 = keras.layers.Input(shape=(3,))
input_2 = keras.layers.Input(shape=(3,))
input_3 = keras.layers.Input(shape=(1,))
initializer = keras.initializers.Ones() #the vector of weights a
l_input_1 = keras.layers.Dense(1,activation='relu',kernel_initializer=initializer,use_bias=False)(input_1)
l_input_2 = keras.layers.Dense(1,activation='relu',use_bias=False,kernel_initializer=initializer,trainable=False)(input_2)
division = keras.layers.Lambda(lambda inputs:  tf.where(inputs[0] != 0, tf.math.minimum(inputs[1]/inputs[0],4), inputs[1]))([l_input_1, l_input_2])#calculates empirical rt
mean = keras.layers.Lambda(lambda x: keras.backend.dot(mat,x))(division) #calculates mean r_seven in seven days
cases = keras.layers.Multiply()([input_3,mean]) #cases for valid prediction,net output, padded with zeros at the end
model = keras.Model(inputs=[input_1,input_2,input_3],outputs=[cases])
optimizer = keras.optimizers.Adam()
model.compile(optimizer, loss=mse_custom)
history = model.fit([m1,m2,a], b, epochs=10000, batch_size=n_dias,shuffle=False)
model.summary()

Epoch 1/10000
350/350 [==============================] - 0s 131us/step - loss: 60755.5664
Epoch 2/10000
350/350 [==============================] - 0s 6us/step - loss: 60723.0781
Epoch 3/10000
350/350 [==============================] - 0s 6us/step - loss: 60727.7266
Epoch 4/10000
350/350 [==============================] - 0s 6us/step - loss: 60731.7695
Epoch 5/10000
350/350 [==============================] - 0s 9us/step - loss: 60724.7930
Epoch 6/10000
350/350 [==============================] - 0s 9us/step - loss: 60714.1094
Epoch 7/10000
350/350 [==============================] - 0s 6us/step - loss: 60706.4922
Epoch 8/10000
350/350 [==============================] - 0s 6us/step - loss: 60703.5820
Epoch 9/10000
350/350 [==============================] - 0s 6us/step - loss: 60702.5430
Epoch 10/10000
350/350 [==============================] - 0s 6us/step - loss: 60699.7539
Epoch 11/10000
350/350 [==============================] - 0s 6us/step - loss: 60694.1953
Epoch 12/10000
350/350 [====

Epoch 94/10000
350/350 [==============================] - 0s 6us/step - loss: 60136.1836
Epoch 95/10000
350/350 [==============================] - 0s 6us/step - loss: 60128.6289
Epoch 96/10000
350/350 [==============================] - 0s 6us/step - loss: 60121.0586
Epoch 97/10000
350/350 [==============================] - 0s 9us/step - loss: 60113.4961
Epoch 98/10000
350/350 [==============================] - 0s 6us/step - loss: 60105.8984
Epoch 99/10000
350/350 [==============================] - 0s 6us/step - loss: 60098.3320
Epoch 100/10000
350/350 [==============================] - 0s 6us/step - loss: 60090.7305
Epoch 101/10000
350/350 [==============================] - 0s 9us/step - loss: 60083.1484
Epoch 102/10000
350/350 [==============================] - 0s 6us/step - loss: 60075.5430
Epoch 103/10000
350/350 [==============================] - 0s 9us/step - loss: 60067.9258
Epoch 104/10000
350/350 [==============================] - 0s 6us/step - loss: 60060.3086
Epoch 105/10000


Epoch 186/10000
350/350 [==============================] - 0s 9us/step - loss: 59431.0586
Epoch 187/10000
350/350 [==============================] - 0s 9us/step - loss: 59423.4414
Epoch 188/10000
350/350 [==============================] - 0s 11us/step - loss: 59415.8242
Epoch 189/10000
350/350 [==============================] - 0s 9us/step - loss: 59408.1953
Epoch 190/10000
350/350 [==============================] - 0s 9us/step - loss: 59400.6055
Epoch 191/10000
350/350 [==============================] - 0s 11us/step - loss: 59392.9883
Epoch 192/10000
350/350 [==============================] - 0s 6us/step - loss: 59385.3711
Epoch 193/10000
350/350 [==============================] - 0s 9us/step - loss: 59377.7539
Epoch 194/10000
350/350 [==============================] - 0s 11us/step - loss: 59370.1719
Epoch 195/10000
350/350 [==============================] - 0s 9us/step - loss: 59362.5781
Epoch 196/10000
350/350 [==============================] - 0s 9us/step - loss: 59354.9883
Epoch 1

Epoch 277/10000
350/350 [==============================] - 0s 6us/step - loss: 58754.4961
Epoch 278/10000
350/350 [==============================] - 0s 9us/step - loss: 58747.3086
Epoch 279/10000
350/350 [==============================] - 0s 6us/step - loss: 58740.0859
Epoch 280/10000
350/350 [==============================] - 0s 9us/step - loss: 58732.9023
Epoch 281/10000
350/350 [==============================] - 0s 6us/step - loss: 58725.6953
Epoch 282/10000
350/350 [==============================] - 0s 6us/step - loss: 58718.5078
Epoch 283/10000
350/350 [==============================] - 0s 6us/step - loss: 58711.3711
Epoch 284/10000
350/350 [==============================] - 0s 6us/step - loss: 58704.1875
Epoch 285/10000
350/350 [==============================] - 0s 6us/step - loss: 58697.0234
Epoch 286/10000
350/350 [==============================] - 0s 9us/step - loss: 58689.8516
Epoch 287/10000
350/350 [==============================] - 0s 9us/step - loss: 58682.7031
Epoch 288/

Epoch 368/10000
350/350 [==============================] - 0s 6us/step - loss: 58123.1641
Epoch 369/10000
350/350 [==============================] - 0s 9us/step - loss: 58116.5273
Epoch 370/10000
350/350 [==============================] - 0s 9us/step - loss: 58109.8672
Epoch 371/10000
350/350 [==============================] - 0s 9us/step - loss: 58103.2383
Epoch 372/10000
350/350 [==============================] - 0s 9us/step - loss: 58096.5898
Epoch 373/10000
350/350 [==============================] - 0s 9us/step - loss: 58089.9766
Epoch 374/10000
350/350 [==============================] - 0s 9us/step - loss: 58083.3672
Epoch 375/10000
350/350 [==============================] - 0s 9us/step - loss: 58076.7500
Epoch 376/10000
350/350 [==============================] - 0s 9us/step - loss: 58070.1328
Epoch 377/10000
350/350 [==============================] - 0s 11us/step - loss: 58063.5430
Epoch 378/10000
350/350 [==============================] - 0s 11us/step - loss: 58056.9414
Epoch 37

...

Epoch 3371/10000
350/350 [==============================] - 0s 6us/step - loss: 53379.1133
Epoch 3372/10000
350/350 [==============================] - 0s 6us/step - loss: 53378.5664
Epoch 3373/10000
350/350 [==============================] - 0s 6us/step - loss: 53378.0039
Epoch 3374/10000
350/350 [==============================] - 0s 6us/step - loss: 53377.4336
Epoch 3375/10000
350/350 [==============================] - 0s 6us/step - loss: 53376.8789
Epoch 3376/10000
350/350 [==============================] - 0s 6us/step - loss: 53376.3125
Epoch 3377/10000
350/350 [==============================] - 0s 9us/step - loss: 53375.7500
Epoch 3378/10000
350/350 [==============================] - 0s 9us/step - loss: 53375.1875
Epoch 3379/10000
350/350 [==============================] - 0s 9us/step - loss: 53374.6289
Epoch 3380/10000
350/350 [==============================] - 0s 20us/step - loss: 53374.0586
Epoch 3381/10000
350/350 [==============================] - 0s 9us/step - loss: 53373.496

Epoch 3461/10000
350/350 [==============================] - 0s 9us/step - loss: 53327.4336
Epoch 3462/10000
350/350 [==============================] - 0s 9us/step - loss: 53326.8359
Epoch 3463/10000
350/350 [==============================] - 0s 6us/step - loss: 53326.2500
Epoch 3464/10000
350/350 [==============================] - 0s 11us/step - loss: 53325.6445
Epoch 3465/10000
350/350 [==============================] - 0s 6us/step - loss: 53325.0586
Epoch 3466/10000
350/350 [==============================] - 0s 9us/step - loss: 53324.4961
Epoch 3467/10000
350/350 [==============================] - 0s 6us/step - loss: 53323.8789
Epoch 3468/10000
350/350 [==============================] - 0s 11us/step - loss: 53323.3047
Epoch 3469/10000
350/350 [==============================] - 0s 9us/step - loss: 53322.6875
Epoch 3470/10000
350/350 [==============================] - 0s 9us/step - loss: 53322.1016
Epoch 3471/10000
350/350 [==============================] - 0s 6us/step - loss: 53321.50

Epoch 3551/10000
350/350 [==============================] - 0s 6us/step - loss: 53273.2812
Epoch 3552/10000
350/350 [==============================] - 0s 9us/step - loss: 53272.6875
Epoch 3553/10000
350/350 [==============================] - 0s 9us/step - loss: 53272.0586
Epoch 3554/10000
350/350 [==============================] - 0s 11us/step - loss: 53271.4414
Epoch 3555/10000
350/350 [==============================] - 0s 9us/step - loss: 53270.8242
Epoch 3556/10000
350/350 [==============================] - 0s 6us/step - loss: 53270.2383
Epoch 3557/10000
350/350 [==============================] - 0s 9us/step - loss: 53269.6055
Epoch 3558/10000
350/350 [==============================] - 0s 9us/step - loss: 53268.9648
Epoch 3559/10000
350/350 [==============================] - 0s 6us/step - loss: 53268.3672
Epoch 3560/10000
350/350 [==============================] - 0s 6us/step - loss: 53267.7500
Epoch 3561/10000
350/350 [==============================] - 0s 9us/step - loss: 53267.125

350/350 [==============================] - 0s 6us/step - loss: 53216.7773
Epoch 3642/10000
350/350 [==============================] - 0s 9us/step - loss: 53216.1602
Epoch 3643/10000
350/350 [==============================] - 0s 9us/step - loss: 53215.4961
Epoch 3644/10000
350/350 [==============================] - 0s 6us/step - loss: 53214.8789
Epoch 3645/10000
350/350 [==============================] - 0s 9us/step - loss: 53214.2461
Epoch 3646/10000
350/350 [==============================] - 0s 9us/step - loss: 53213.5820
Epoch 3647/10000
350/350 [==============================] - 0s 7us/step - loss: 53212.9375
Epoch 3648/10000
350/350 [==============================] - 0s 6us/step - loss: 53212.3086
Epoch 3649/10000
350/350 [==============================] - 0s 6us/step - loss: 53211.6289
Epoch 3650/10000
350/350 [==============================] - 0s 6us/step - loss: 53211.0039
Epoch 3651/10000
350/350 [==============================] - 0s 9us/step - loss: 53210.3789
Epoch 3652/10000

Epoch 3732/10000
350/350 [==============================] - 0s 6us/step - loss: 53157.3320
Epoch 3733/10000
350/350 [==============================] - 0s 6us/step - loss: 53156.6641
Epoch 3734/10000
350/350 [==============================] - 0s 6us/step - loss: 53156.0039
Epoch 3735/10000
350/350 [==============================] - 0s 6us/step - loss: 53155.3477
Epoch 3736/10000
350/350 [==============================] - 0s 6us/step - loss: 53154.6797
Epoch 3737/10000
350/350 [==============================] - 0s 9us/step - loss: 53154.0039
Epoch 3738/10000
350/350 [==============================] - 0s 9us/step - loss: 53153.3125
Epoch 3739/10000
350/350 [==============================] - 0s 6us/step - loss: 53152.6641
Epoch 3740/10000
350/350 [==============================] - 0s 9us/step - loss: 53151.9883
Epoch 3741/10000
350/350 [==============================] - 0s 6us/step - loss: 53151.3203
Epoch 3742/10000
350/350 [==============================] - 0s 6us/step - loss: 53150.6641

In [ ]:
res = model.predict([m1,m2,a],batch_size=n_dias)

In [ ]:
first_layer_weights = model.layers[2].get_weights()
second_layer_weights = model.layers[3].get_weights()
print(first_layer_weights,second_layer_weights)

In [ ]:
plt.plot(history.history['loss'])

In [ ]:
res = model.predict([m1,m2,a],batch_size=n_dias)

In [ ]:
plt.plot(res[3:-3],label='result')
plt.plot(b[3:-3],label='groundtruth')
plt.legend()